# Inference pipeline for Chiayu 

This page shows how to run a trained model to get its prediction \
What is needed is:
* the model file in pt 
* the config.json for the initialization of the model architecture
* the vocab and merges file for the tokenizer -- we will then add our tokens inside so i dont have more files to deal with

In [5]:
from transformers import AutoTokenizer, AutoModelForPreTraining ,AutoConfig
import torch
import time

In [7]:
model_dir = './colab_output/'
model_output = model_dir + 'pretty-lion-15.pt'


SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}

tokenizer = AutoTokenizer.from_pretrained(model_dir) 
tokenizer.add_special_tokens(SPECIAL_TOKENS)

model_config = AutoConfig.from_pretrained(model_dir+'config.json', 
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)

model = AutoModelForPreTraining.from_pretrained(model_output, config=model_config)



In [15]:
#device = 'cpu' 
device = 'cpu'

rating = 'boring, not funny, not funny, horrible'

def gen_suggestions(rating,model):
    prompt = SPECIAL_TOKENS['bos_token'] + rating + SPECIAL_TOKENS['sep_token']
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
    model.to(device)
    model.eval();

      # Top-p (nucleus) text generation (10 samples):
    sample_outputs = model.generate(generated, 
                                      do_sample=True,   
                                      min_length=50, 
                                      max_length=128,
                                      top_k=10,                                 
                                      top_p=0.7,        
                                      temperature=0.9,
                                      repetition_penalty=2.0,
                                      num_return_sequences=1
                                      )
    texts = []
    for i, sample_output in enumerate(sample_outputs):
            text = tokenizer.decode(sample_output, skip_special_tokens=True)   
            texts.append(text[len(rating):])
    print(texts[0])
start_time = time.time()
gen_suggestions('boring, not funny, not funny, horrible',model)
end_time = time.time()
print(f"\n\n------ Processing time is: {end_time-start_time}s")

I've been going to this location several times and every time I order a pizza from here the crust is not even warm. The toppings are thin on top of each other. They have a weird sauce they put in the pizza which is really weird for me. It was good but I was disappointed.


------ Processing time is: 4.392829656600952s


#to deploy to gcloud

gcloud builds submit --tag gcr.io/PROJECT-ID/SERVICE-NAME \
gcloud run deploy --image gcr.io/PROJECT-ID/SERVICE-NAME --platform managed